<a href="https://colab.research.google.com/github/lucasplandim/imersao_ai/blob/main/Projeto_ai_imers%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este projeto visa auxiliar as pessoas em suas apostas esportivas. Elas irão informar o confronto desejado e o modelo irá trazer detalhes e estatísticas deste confronto. Além disso, o usuário poderá ter mais detalhes do confronto em questão como, por exemplo, saber quem é o maior goleador de cada time, qual o retrospecto do time jogando em casa, assim por diante.

Instalando o SDK do Google

In [ ]:
!pip install -q -U google-generativeai

Importando a library do Google Generative AI

In [ ]:
import google.generativeai as genai

Configurando a API Key para conseguirmos acessar as APIs do Gemini

*Crie a sua API Key através deste link [Google AI Studio > API Key](https://aistudio.google.com/app/apikey/?utm_source=website&utm_medium=referral&utm_campaign=Alura&utm_content=)*

In [ ]:
from google.colab import userdata
api_key = userdata.get('api_key')

genai.configure(api_key=api_key)

Setando as configurações do modelo e inicializando-o

In [ ]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5
}

safety_settings = {
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
}

system_instruction = {"""Você é um assistente virtual que irá auxiliar as pessoas em suas apostas esportivas.
A pessoa irá informar o confronto desejado e você terá que trazer vários detalhes e estatísticas deste confronto.
É importante dizer qual o retrospecto de um time com o outro. Quantas vitórias, derrotas e empates.
Além disso, se atente ao ano que estamos no momento e traga as informações mais recentes.
Ao final da primeira análise você deve sugerir um palpite para o placar do jogo."""}

model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                              generation_config=generation_config,
                              safety_settings=safety_settings,
                              system_instruction = system_instruction)

Inciando o chat

In [ ]:
chat = model.start_chat(history=[])

# Pega a entrada do usuário
print('Qual jogo você deseja saber os detalhes/estatísticas para te auxiliar nas suas apostas?')
print('Coloque sua resposta no padrão >> Time1 x Time2')
print('(Ou digite "fim" para encerrar o chat)')
prompt = input()

# Função usada para validar a entrada
def validar_entrada():

  if prompt.strip() == "":
    raise ValueError("Nada foi passado na entrada.")

  while True:
    times = prompt.split(" x ")
    if len(times) != 2:
        raise ValueError("Formato inválido. Digite >> Time1 x Time2")
    return times

# Função principal
def main():
    global prompt
    count = 0

    while prompt != "fim":
        try:
            # Validação de entrada apenas na primeira interação
            if count == 0:
                times = validar_entrada()

            # Envio da mensagem e tratamento da resposta
            response = chat.send_message(prompt)
            exibir_detalhes(response)

            # Prompt para continuar ou finalizar
            print("Deseja saber mais algum detalhe deste confronto?\nCaso não precise de mais nenhuma informação, digite 'fim' para encerrar o chat: ")
            prompt = input()
            count += 1

        except ValueError as e:
            print(f"Erro: {e}")
            print("Tente novamente ou digite 'fim' para encerrar.")
            prompt = input()

def exibir_detalhes(response):
    print('Detalhes do confronto:')
    print(response.text, '\n\n')

if __name__ == "__main__":
    main()

print("Obrigado por usar o chat! Volte sempre.")

Melhorando a visualização

In [ ]:
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#Imprimindo o histórico
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
  print('-------------------------------------------')